# IR HW7 基于知识图谱的问答系统

In [2]:
from ltp import LTP
ltp = LTP()

In [3]:
from py2neo import Graph, Node, Relationship

graph = Graph("http://localhost:7474", username='neo4j', password='Qazwsxedcrfv0957')

In [65]:
class Extractor:
    def __init__(self):
        global ltp
        global graph
        self.outpath='test_result/'
        self.temp_outpath=self.outpath

    def read_file(self, path):
        """
        读取一个文档
        """
        self.temp_outpath+=path
        with open(path, 'r',encoding='utf-8') as fr:
            self.segs, self.hidden = ltp.seg(fr.read().split(' '))
            self.srls = ltp.srl(self.hidden, keep_empty=False)
            assert len(self.segs) == len(self.srls)
            
    def execute_instr(self,path):
        """
        执行已经生成的指令，以将节点添加到数据库
        """
        with open(path, 'r',encoding='utf-8') as fr:
            graph.run(fr.readline())

    def concat_str(self, seg, triple: tuple):
        """
        Parameters
        --------
        :param triple: tuple, ('ARG2', 3, 5)
        """
        return ''.join(seg[triple[1]:(triple[2] + 1)])

    def extract(self):
        """
        完成对一篇文章的分析，将生成的指令先存储到文件，之后再用cypher语言批处理，以提高效率
        """
        with open(self.temp_outpath,'w',encoding='utf-8') as fw:
            for i in range(len(self.segs)):
                seg = self.segs[i]
                srl = self.srls[i]
                for tup in srl:
                    relation_name = seg[tup[0]]
                    so_list = tup[1]
                    s = []
                    o = []
                    properties = []
                    for arg in so_list:
                        if arg[0] == 'A0':
                            s.append(arg)
                        elif 'ARGM' not in arg[0]:
                            o.append(arg)
                        else:
                            properties.append(arg)

                    # 生成关系语句
                    for node1 in s:
                        for node2 in o:
                            instr="merge (m:" + node1[0] + "{name:\'" + self.concat_str(seg,node1) + "\'})-[r:" + relation_name + "]-(n:" +node2[0] + "{name:\'" + self.concat_str(seg, node2) + "\'}) ON CREATE SET r.value=1 on match set r.value=r.value+1;\n"
                            fw.write(instr)
        self.temp_outpath=self.outpath
            
    def retrieval(self,question:str):
        """
        查询
        """
        segs, hidden = ltp.seg([question])
        srls = ltp.srl(hidden, keep_empty=False)
        assert len(segs) == len(srls)
        for i in range(len(segs)):
            seg = segs[i]
            srl = srls[i]
            for tup in srl:
                relation_name = seg[tup[0]]
                so_list = tup[1]
                s = []
                o = []
                properties = []
                res=[]
                for arg in so_list:
                    if arg[0] == 'A0':
                        s.append(arg)
                    elif 'ARGM' not in arg[0]:
                        o.append(arg)
                    else:
                        properties.append(arg)
                if not s and not o:
                    print('ask another question')
                    return;
                if o and not s:
                    for oo in o:
                        res+=graph.run("match  (n:" + oo[0] + "{name:\'" + self.concat_str(seg,oo) + "\'})-[r:"+relation_name+"]-(m) return m;").data()
                elif s and not o:
                    for ss in s:
                        res+=graph.run("match  (n:" + ss[0] + "{name:\'" + self.concat_str(seg,ss) + "\'})-[r:"+relation_name+"]-(m) return m;").data()
                elif s and o:
                    for ss in s:
                        for oo in o:
                            res+=graph.run("match  (n:" + oo[0] + "{name:\'" + self.concat_str(seg,oo) + "\'})-[r:"+relation_name+"]-(m) return m;").data()
                        res+=graph.run("match  (n:" + ss[0] + "{name:\'" + self.concat_str(seg,ss) + "\'})-[r:"+relation_name+"]-(m) return m;").data()
                        
                print(res)

                
                

        


In [69]:
import os
e=Extractor()
for filepath, dirs, fs in os.walk('test_page/2021_1_2/'):
    for filename in fs:
        fullpath = os.path.join(filepath, filename)
        print(fullpath)
        e.read_file(fullpath)
        e.extract()

test_page/2021_1_2/348.txt
test_page/2021_1_2/350.txt
test_page/2021_1_2/354.txt
test_page/2021_1_2/355.txt
test_page/2021_1_2/356.txt
test_page/2021_1_2/357.txt
test_page/2021_1_2/358.txt
test_page/2021_1_2/359.txt
test_page/2021_1_2/360.txt


In [70]:
import os
e=Extractor()
for filepath, dirs, fs in os.walk('test_page/selected/'):
    for filename in fs:
        fullpath = os.path.join(filepath, filename)
        e.execute_instr(fullpath)

In [71]:
e=Extractor()
e.retrieval('南开大学是什么大学')

[{'m': Node('A1', name='国内学科门类齐全的综合性、研究型大学之一')}]
